# Что, где, когда?

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt

## 1. Обработка данных

In [3]:
players = pd.read_pickle('/content/drive/MyDrive/adv-ml/players.pkl')
tournaments = pd.read_pickle('/content/drive/MyDrive/adv-ml/tournaments.pkl')
results = pd.read_pickle('/content/drive/MyDrive/adv-ml/results.pkl')

In [4]:
# Превью загруженных данных
print('Игроки')
print(players[1], players[22799])
print('Турниры')
print(tournaments[1], tournaments[2])
print('Результаты')
print(results[1], results[2])

Игроки
{'id': 1, 'name': 'Алексей', 'patronymic': None, 'surname': 'Абабилов'} {'id': 22799, 'name': 'Сергей', 'patronymic': 'Игоревич', 'surname': 'Николенко'}
Турниры
{'id': 1, 'name': 'Чемпионат Южного Кавказа', 'dateStart': '2003-07-25T00:00:00+04:00', 'dateEnd': '2003-07-27T00:00:00+04:00', 'type': {'id': 2, 'name': 'Обычный'}, 'season': '/seasons/1', 'orgcommittee': [], 'synchData': None, 'questionQty': None} {'id': 2, 'name': 'Летние зори', 'dateStart': '2003-08-09T00:00:00+04:00', 'dateEnd': '2003-08-09T00:00:00+04:00', 'type': {'id': 2, 'name': 'Обычный'}, 'season': '/seasons/1', 'orgcommittee': [], 'synchData': None, 'questionQty': None}
Результаты
[{'team': {'id': 242, 'name': 'Команда Азимова', 'town': {'id': 21, 'name': 'Баку'}}, 'mask': None, 'current': {'name': 'Команда Азимова', 'town': {'id': 21, 'name': 'Баку'}}, 'questionsTotal': 0, 'synchRequest': None, 'position': 1, 'controversials': [], 'flags': [], 'teamMembers': [{'flag': None, 'usedRating': 0, 'rating': 0, 'pl

In [5]:
players_df = pd.DataFrame.from_dict(players, orient='index')
tournaments_df = pd.DataFrame.from_dict(tournaments, orient='index')
results_df = pd.DataFrame.from_dict(results, orient='index')

In [6]:
tournaments_df['dateStart'] = pd.to_datetime(tournaments_df['dateStart'], utc=True)
tournaments_df['dateEnd'] = pd.to_datetime(tournaments_df['dateEnd'], utc=True)

In [7]:
players_df.head(3)

,id,name,patronymic,surname
1,1,Алексей,None,Абабилов
10,10,Игорь,,Абалов
11,11,Наталья,Юрьевна,Абалымова


In [8]:
tournaments_df.head(3)

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
1,1,Чемпионат Южного Кавказа,2003-07-24 20:00:00+00:00,2003-07-26 20:00:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,2,Летние зори,2003-08-08 20:00:00+00:00,2003-08-08 20:00:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,3,Турнир в Ижевске,2003-11-21 21:00:00+00:00,2003-11-23 21:00:00+00:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


### Отбор

In [9]:
train_tournaments = tournaments_df[(tournaments_df['dateStart'] >= '2019-01-01') & (tournaments_df['dateStart'] <= '2019-12-31')]
test_tournaments = tournaments_df[(tournaments_df['dateStart'] >= '2020-01-01') & (tournaments_df['dateStart'] <= '2020-12-31')]

In [10]:
def clean_tournaments(tournaments: pd.DataFrame, results: pd.DataFrame) -> pd.DataFrame:
  cleaned_tournaments = tournaments.copy()
  for id in cleaned_tournaments['id']:
    if not results.loc[id, 0] or ('mask' in results.loc[id, 0].keys() and not results.loc[id, 0]['mask']):
      cleaned_tournaments.drop([id], inplace=True)
  
  return cleaned_tournaments

In [11]:
train_tournaments_cleaned = clean_tournaments(train_tournaments, results_df)
test_tournaments_cleaned = clean_tournaments(test_tournaments, results_df)

In [12]:
print('Shape train до чистки:', train_tournaments.shape, 'после', train_tournaments_cleaned.shape)
print('Shape test до чистки:', test_tournaments.shape, 'после', test_tournaments_cleaned.shape)

Shape train до чистки: (687, 9) после (675, 9)
Shape test до чистки: (415, 9) после (191, 9)


### Вспомогательные функции

## 2. Baseline-модель

* Baseline-модель на основе логистической регрессии.
* Повопросные результаты команды относятся к каждому из её игроков.
* Учитываем сложность вопросов (по уровню сложности турнира).

In [ ]:
# Готовим матрицу
def create_matrix(tournaments: pd.DataFrame):
